In [1]:
# 데이터 다운로드 링크로 데이터를 코랩에 불러옵니다.

!wget 'https://bit.ly/3gLj0Q6'

import zipfile
with zipfile.ZipFile('3gLj0Q6', 'r') as existing_zip:
    existing_zip.extractall('data')

--2021-09-07 14:57:34--  https://bit.ly/3gLj0Q6
Resolving bit.ly (bit.ly)... 67.199.248.10, 67.199.248.11
Connecting to bit.ly (bit.ly)|67.199.248.10|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://drive.google.com/uc?export=download&id=1or_QN1ksv81DNog6Tu_kWcZ5jJWf5W9E [following]
--2021-09-07 14:57:35--  https://drive.google.com/uc?export=download&id=1or_QN1ksv81DNog6Tu_kWcZ5jJWf5W9E
Resolving drive.google.com (drive.google.com)... 142.251.2.113, 142.251.2.139, 142.251.2.138, ...
Connecting to drive.google.com (drive.google.com)|142.251.2.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0c-10-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ju2ei4g11f68i0hqg2mdlals43fak44r/1631026650000/17946651057176172524/*/1or_QN1ksv81DNog6Tu_kWcZ5jJWf5W9E?e=download [following]
--2021-09-07 14:57:35--  https://doc-0c-10-docs.googleusercontent.com/docs/securesc/ha0r

In [2]:
import pandas as pd

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
# 결측치가 있는 feature 확인
print(train.isnull().sum())

id                          0
hour                        0
hour_bef_temperature        2
hour_bef_precipitation      2
hour_bef_windspeed          9
hour_bef_humidity           2
hour_bef_visibility         2
hour_bef_ozone             76
hour_bef_pm10              90
hour_bef_pm2.5            117
count                       0
dtype: int64


In [4]:
# 평균값으로 결측치 대체하기
train.fillna({'hour_bef_temperature':int(train['hour_bef_temperature'].mean())}, inplace=True)
train.fillna({'hour_bef_precipitation':int(train['hour_bef_precipitation'].mean())}, inplace=True)
train.fillna({'hour_bef_windspeed':int(train['hour_bef_windspeed'].mean())}, inplace=True)
train.fillna({'hour_bef_humidity':int(train['hour_bef_humidity'].mean())}, inplace=True)
train.fillna({'hour_bef_visibility':int(train['hour_bef_visibility'].mean())}, inplace=True)
train.fillna({'hour_bef_ozone':int(train['hour_bef_ozone'].mean())}, inplace=True)
train.fillna({'hour_bef_pm10':int(train['hour_bef_pm10'].mean())}, inplace=True)
train.fillna({'hour_bef_pm2.5':int(train['hour_bef_pm2.5'].mean())}, inplace=True)

print(train.isnull().sum())

id                        0
hour                      0
hour_bef_temperature      0
hour_bef_precipitation    0
hour_bef_windspeed        0
hour_bef_humidity         0
hour_bef_visibility       0
hour_bef_ozone            0
hour_bef_pm10             0
hour_bef_pm2.5            0
count                     0
dtype: int64


In [5]:
# 보간법을 사용하여 결측치 대체하기
train.interpolate(inplace=True)

print(train.isnull().sum())

id                        0
hour                      0
hour_bef_temperature      0
hour_bef_precipitation    0
hour_bef_windspeed        0
hour_bef_humidity         0
hour_bef_visibility       0
hour_bef_ozone            0
hour_bef_pm10             0
hour_bef_pm2.5            0
count                     0
dtype: int64


In [4]:
# (진) 결측치 전처리
train.interpolate(inplace=True)
test.fillna(0, inplace=True)

In [5]:
X_train = train.drop(['count'], axis=1)
Y_train = train['count']

# 랜덤포레스트
여러개의 의사결정나무(decisiontree)를 만들어 이들의 평균으로 예측 성능을 높이는 방법.

이러한 기법을 앙상블(Ensemble)이라고 함.

In [6]:
# 랜덤포레스트 회귀 모델 선언하기
from sklearn.ensemble import RandomForestRegressor

# criterion 옵션: 어떤 평가 척도를 기준으로 훈련할 것인지 정함
model = RandomForestRegressor(criterion='mse')

In [7]:
model.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [8]:
# 변수 중요도
# 예측 변수를 결정할 때 각 피처가 얼마나 중요한 역할을 하는지에 대한 척도

print(model.feature_importances_)

[0.02567989 0.59304244 0.17722764 0.01940117 0.02513299 0.03347477
 0.03362094 0.0381967  0.03239969 0.02182377]


In [10]:
X_train_1 = train.drop(['count', 'id'], axis=1)
X_train_2 = train.drop(['count', 'id', 'hour_bef_windspeed'], axis=1)
X_train_3 = train.drop(['count', 'id', 'hour_bef_windspeed', 'hour_bef_pm2.5'], axis=1)

In [12]:
test_1 = test.drop(['id'], axis=1)
test_2 = test.drop(['id', 'hour_bef_windspeed'], axis=1)
test_3 = test.drop(['id', 'hour_bef_windspeed', 'hour_bef_pm2.5'], axis=1)

In [13]:
model_1 = RandomForestRegressor(criterion='mse')
model_1.fit(X_train_1, Y_train)

model_2 = RandomForestRegressor(criterion='mse')
model_2.fit(X_train_2, Y_train)

model_3 = RandomForestRegressor(criterion='mse')
model_3.fit(X_train_3, Y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [15]:
pred_1 = model_1.predict(test_1)
pred_2 = model_2.predict(test_2)
pred_3 = model_3.predict(test_3)

In [17]:
submission_1 = pd.read_csv('data/submission.csv')
submission_2 = pd.read_csv('data/submission.csv')
submission_3 = pd.read_csv('data/submission.csv')

submission_1['count'] = pred_1
submission_2['count'] = pred_2
submission_3['count'] = pred_3

submission_1.to_csv('sub_1.csv', index=False)
submission_2.to_csv('sub_2.csv', index=False)
submission_3.to_csv('sub_3.csv', index=False)